In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

### add command path

In [2]:
import sys,numpy,pandas,os
from copy import deepcopy
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')

### import functions

In [20]:
from command.nexus.SPheno import SPheno,block_mapping
from command.scan.scan import scan
from command.multi_thread.MT_SPheno import MT_SPheno
from command.multi_thread.queue_operation import GenerateQueue # generate random queue
from command.pytorch.normalize import normalize
from command.operators.iterable import FlatToList
from command.data_transformer.ArrayToInputQueue import ArrayToInputQueue
from command.data_transformer.InputToPandas import InputToPandas

In [4]:
mold=scan(method='random')
# MinPar
mold.AddScalar('tanB','MINPAR',3,1.,60.)
# Extpar
mold.AddScalar('Lambda','EXTPAR'  ,61  ,1e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('Kappa','EXTPAR'   ,62 ,1.e-3    ,1. ,prior_distribution='exponential')
mold.AddScalar('A_Lambda','EXTPAR' ,63,-3.e3,3.e3)
mold.AddScalar('A_kappa','EXTPAR' ,64,-3.e3,3.e3)
mold.AddScalar('mu_eff','EXTPAR'  ,65,100.,1500.)
# Squark mass
mold.AddElement( 'Mq32','MSQ2',(3,3),1e4,4e6)
mold.AddElement( 'Mu32','MSU2',(3,3),1e4,4e6)
mold.AddFollower('Md32','MSD2',(3,3),'Mu32')
# Slepton mass
mold.AddElement( 'Ml32','MSL2',(3,3),1e4,4e6)
mold.AddFollower('Ml22','MSL2',(2,2),'Ml32')
mold.AddFollower('Ml12','MSL2',(1,1),'Ml32')
mold.AddFollower('Me32','MSE2',(3,3),'Ml32')
mold.AddFollower('Me22','MSE2',(2,2),'Ml32')
mold.AddFollower('Me12','MSE2',(1,1),'Ml32')
# Yukawa
mold.AddElement( 'Tt' ,'TU'  ,(3,3),-2000,2000)
mold.AddElement( 'Tb' ,'TD'  ,(3,3),-2000,2000)
mold.AddElement( 'Ttau' ,'TE'  ,(3,3),-2000,2000)
# MSoft
mold.AddScalar('M1','MSOFT',1,50,2000)
mold.AddScalar('M2','MSOFT',2,100,2000)

In [5]:
mold.GetValue('LesHouches.in.NMSSM_low',mapping=block_mapping)
mold.Print()

   A_kappa: -36.0
  A_Lambda: -1500.0
      Ml12: 40000.0
    mu_eff: 965.0
    Lambda: 0.1
     Kappa: 0.11
      Me32: 40000.0
      Ml32: 40000.0
      Me22: 40000.0
        Tt: 1000.0
      Ml22: 40000.0
      Md32: 1000000.0
      Me12: 40000.0
        Tb: 1000.0
        M2: 2000.0
      Ttau: 0.0
      tanB: 10.0
      Mu32: 1000000.0
        M1: 500.0
      Mq32: 1000000.0


In [6]:
order=FlatToList(map(lambda x: list(x.keys()),mold.free_parameter_list.maps))
order

['tanB',
 'Lambda',
 'Kappa',
 'A_Lambda',
 'A_kappa',
 'mu_eff',
 'M1',
 'M2',
 'Mq32',
 'Mu32',
 'Ml32',
 'Tt',
 'Tb',
 'Ttau']

In [7]:
dimention=len(order)
dimention

14

### normalize & tabulate parameters from mold list to numpy array

In [10]:
Norm=normalize(mold,order=order) # set operation scale=(a,b) to scale parameters to (a,b) range
Norm.scale

(0, 1)

### make a input point queue
*choose one*

In [8]:
# Random points queue by scan's function: Sample
RandSamples=GenerateQueue(mold,lenth=1000) 

In [19]:
# # Random points queue from numpy.random array
# normal_array=numpy.random.rand(1000,dimention) 
# par_array=Norm.D(normal_array)
# RandSamples=ArrayToInputQueue(par_array,mold,order=order)

### multithread SPheno

In [8]:
MTS=MT_SPheno(threads=6,Renew=False)

SPheno file:
->/home/heyangle/Desktop/ScanCraft/ScanCraft/packages/SPheno-4.0.3


In [9]:
MTS.Run(RandSamples,report_interval=1000)#,ReMake='NMSSM_sarah')

Calculations begin at Tue Jun 26 17:05:34 2018
  threads:	6
  points:	1000
Thread-3 got 0 samples
Thread-0 got 0 samples
Thread-2 got 0 samples
Thread-5 got 0 samples
Thread-4 got 0 samples
Thread-1 got 0 samples
  thread-5	runing,        0 points left at Tue Jun 26 17:16:17 2018
All points done. Use 0.183557 hours


KeyError: 'spectrum'

In [64]:
for k,v in mold.variable_list.items():
    print(k,v.value)

Lambda 0.1
mu_eff 965.0
tanB 10.0
Mq32 1000000.0
Tt 1000.0
Md32 1000000.0
A_Lambda -1500.0
M1 500.0
Ml22 40000.0
Ml12 40000.0
Ttau 0.0
A_kappa -36.0
Mu32 1000000.0
Ml32 40000.0
Me32 40000.0
Me22 40000.0
Me12 40000.0
Tb 1000.0
M2 2000.0
Kappa 0.11


In [73]:
from collections import namedtuple
from command.data_transformer.defult_parameter_order import defult_name_order

In [38]:
ID=namedtuple('identity_tuple',['block','code','name'])

In [44]:
l=ID('E',63,'Lambda')

In [79]:
defult_name_order(mold.free_parameter_list)

63 64 A_Lambda A_kappa
65 63 mu_eff A_Lambda
61 65 Lambda mu_eff
62 61 Kappa Lambda
6 1 Ml32 Kappa
7 6 Tt Ml32
8 7 Tb Tt
2 8 M2 Tb
2 1 M2 Kappa
2 7 M2 Tt
2 6 M2 Ml32
9 1 Ttau Kappa
9 7 Ttau Tt
9 8 Ttau Tb
0 2 tanB M2
0 1 tanB mu_eff
0 1 tanB A_Lambda
0 1 tanB A_kappa
4 1 Mu32 Kappa
4 7 Mu32 Tt
4 6 Mu32 Ml32
4 2 Mu32 M2
1 2 M1 M2
2 7 M1 Tt
2 6 M1 Ml32
2 4 M1 Mu32
3 2 Mq32 M2
3 7 Mq32 Tt
3 4 Mq32 Mu32
3 2 Mq32 M1


['tanB',
 'A_kappa',
 'A_Lambda',
 'mu_eff',
 'Lambda',
 'Kappa',
 'M2',
 'M1',
 'Mq32',
 'Mu32',
 'Ml32',
 'Tt',
 'Tb',
 'Ttau']

In [50]:
l[0]

'E'

In [68]:
class ID(namedtuple('identity_tuple',['block','code','name'])):
    __slots__=()
    def __str__(self):
        return f'{self.name:>10}:{self.block:>10}: {self.code}'

In [69]:
l=ID('E',63,'Lambda')

In [70]:
str(l)

'    Lambda:         E: 63'

In [72]:
'1'<'a'

True